# Walkthrough the NIRCAM imaging mode  WCS pipeline

### Make sure that you have set the JWST_NOTEBOOK_DATA environment variable in the terminal from which you started Jupyter Notebook.

The data will be read from that directory, and the pipeline should write to the current working directory, avoiding clobbers.
If you would like to use your own data just substitute the locations below.

In [ ]:
# plotting, the inline must come before the matplotlib import
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.patches as patches

params = {'legend.fontsize': 6,
          'figure.figsize': (8, 8),
          'figure.dpi': 150,
         'axes.labelsize': 6,
         'axes.titlesize': 6,
         'xtick.labelsize':6,
         'ytick.labelsize':6}
plt.rcParams.update(params)


# python general
import os
import numpy as np

# astropy modules
import astropy
from astropy.io import fits
from astropy.table import QTable
from astropy.wcs.utils import skycoord_to_pixel
import photutils

# jwst 
import jwst
from jwst.source_catalog import source_catalog_step
from jwst.datamodels import image, DrizProductModel
from jwst import assign_wcs
from jwst.assign_wcs import assign_wcs_step

In [ ]:
import jwst
jwst.__version__

In [ ]:
notebook_dir = os.environ['JWST_NOTEBOOK_DATA']
nircam_data = notebook_dir + 'nircam/'

In [ ]:
# add in the columns for ra and dec min/max points, translated from the wcs object for now
direct_data= nircam_data + 'test_disperse_f335m_rate_updated.fits'   # original image provided for testing

# We will open the direct image as an Image datamodel
direct_image = image.ImageModel(direct_data)

### Some basics about this image

In [ ]:
direct_image.meta.instrument.pupil, direct_image.meta.instrument.filter, direct_image.meta.instrument.module, direct_image.meta.instrument.detector, direct_image.meta.instrument.channel

### This is the FITS WCS information

In [ ]:
direct_image.get_fits_wcs()

In [ ]:
# Load up the assign_wcs step that will assign all the transforms from world->detector
assign_wcs_step=assign_wcs.AssignWcsStep()
reference_file_types = ['distortion', 'filteroffset', 'specwcs', 'regions',
                            'wavelengthrange', 'camera', 'collimator',
                            'disperser', 'fore', 'fpa', 'msa', 'ote', 'ifupost',
                            'ifufore', 'ifuslicer']
reference_file_names = {}

# Ask CRDS for the reference files that apply to the image are working with
for name in reference_file_types:
    reffile = assign_wcs_step.get_reference_file(direct_image, name)
    reference_file_names[name] = reffile if reffile else ""
reference_file_names

In [ ]:
direct_gwcs = assign_wcs_step(direct_image)

### The GWCS object that contains all the transforms is now attached to the image model

In [ ]:
direct_gwcs.meta.wcs

### Check the transform from detector pixels to sky coordinates in decimal degrees of RA and DEC

The default transform goes from detector pixels to sky coordinate (ra,dec)

In [ ]:
direct_gwcs.meta.wcs(110,110)

In [ ]:
detector_to_world = direct_gwcs.meta.wcs.get_transform('detector','world')
detector_to_world(110,110)

### Now get the inverse transform from RA,DEC  to detector pixels, using the RA,DEC we just calculated
This should return the pixel (110,110)

In [ ]:
world_to_detector = direct_gwcs.meta.wcs.get_transform('world','detector')
world_to_detector(0.00804448203007923, -0.007899731808577077)